# Team Compositions in Professional League of Legends
***

The data being used was obtained from [Oracle's Elixer] (using upload from September 18th, 2017). A [codebook] for the variables is also included.

- **Part 1: Data Cleaning** -- can be found [here](http://nbviewer.jupyter.org/github/gmlander/Springboard/blob/master/Portfolio/Team_Composition_Analysis/I_Data_Cleaning.ipynb) or at the raw [github version](https://github.com/gmlander/Springboard/blob/master/Portfolio/Team_Composition_Analysis/I_Data_Cleaning.ipynb).
- **Part 2: Exploratory Data Analysis** -- can be found [here](http://nbviewer.jupyter.org/github/gmlander/Springboard/blob/master/Portfolio/Team_Composition_Analysis/II_Exploratory_Data_Analysis.ipynb) or at the [github version](https://github.com/gmlander/Springboard/blob/master/Portfolio/Team_Composition_Analysis/II_Exploratory_Data_Analysis.ipynb)

[Oracle's Elixer]: http://oracleselixir.com/match-data/
[codebook]: http://oracleselixir.com/match-data/match-data-dictionary/

---

# Part 3: Feature Engineering

#### Prepare Workspace and Get Data

The data being used was prepared in **Part 2**.

In [1]:
%%capture
# To keep these documents from getting untidy
# support_functions.py in this repo containts
# all necessary package imports and custom functions
from support_functions import *
%matplotlib inline

In [2]:
playerDF = unpickle('data/eda_playerDF.pickle')
teamDF = unpickle('data/eda_teamDF.pickle')

## Champion Referrence Table

Champions can be played differently by patch, position, and game. Using one hot encoded champions as features for a logistical regression is not only ineffective and uninformative, it's also illogical. Since the challenge is to get features for observations that don't reveal anything about in-game happenings, the best way to achieve this will be by deriving aggregate information on champions that can then aid in clustering them through unsupervised learning algorithms.

These features will be result-agnostic and performed through four main stages.

1. **Evaluate Champion Frequency**
    - Grouping by patch and position, look at the distributions of champion selections for each patch-position combination.
    - Use this information to create a categorical variable, pick-level, that will capture the prominance of a champion in each patch-position -- meta, low-priority, or off-meta.
2. **Capture Champion Averages**
    - Grouping the data by patch, position, and champion, capture things like average win-rate, average win-time, average damage to champions per minute, etc.
3. **Calculate Average Win and Loss Times**
    - Additional information to help understand champion power spikes.
3. **Build Position Specific Features**
    - Similar to 2., derive features that have specific importance in categorizing each position. For instance, AD-Carry performance as a feature for supports, own/enemy jungle clear rate for junglers, first mid outer turret for mid laners, etc.

### Phase 1: Evaluate Champion Frequency
---

**Get pick counts**

Group by patchno and position, take the value counts of champion and assign it to `patch_df`.

In [3]:
f_players = pd.concat([playerDF[k] for k in playerDF]).sort_index()
patch_df = f_players.groupby(['patchno', 'position'])['champion'].value_counts().to_frame()
patch_df.rename(columns = {'champion':'picks'}, inplace = True)
patch_df.sample()

,,,picks
patchno,position,champion,
7.04,Support,Thresh,23


**Get Ban Count and Frequency**

Because champion bans can say just as much about what players prioritize as selections, it's important to capture this information as well. I'll melt the ban selections from teamDF, group by patchno and position, take the value counts and store them in bans df.

In [4]:
bans = pd.melt(teamDF[teamDF.league != 'LPL'], id_vars=['patchno'], \
            value_vars=['ban'+str(n) for n in range(1,6)], value_name='champion')
bans.drop('variable', axis = 1, inplace = True)
bans.dropna(inplace = True)
bans['patchno'] = bans.patchno.astype(str)

bans = bans.groupby('patchno')['champion'].value_counts()
bans = bans.to_frame()
bans.rename(columns={'champion':'ban_count'}, inplace = True)

bans['ban_percent'] = bans.ban_count / bans.groupby(level='patchno')['ban_count'].transform(sum)
bans.head()

ban_count  ban_percent
patchno champion                          
5.24    Gangplank          33     0.152778
        Ryze               27     0.125000
        Lulu               24     0.111111
        Kindred            14     0.064815
        Tahm Kench         14     0.064815

**Ban Count Distribution**

---

Ultimately ban counts will be added to the pick counts. Unfortunately, because so many champions can be played in multiple positions, and the ban features can't possibly say which position a champion was meant to be banned for, there needs to be a way of adding ban values to multiple positions.

To keep it simple, bans will be added proportionately to picks. So if Shen was banned 20 times, and picked 10 times for support, 40 times for top -- Shen's weighted picks will be 14 for support and 56 for top.

This gets a little messy with index resets and unstacking, but basically it gets the picks by position in `patch_df`, determines the totals for each patchno-champion, and divides by the total to get the pick-rate by position.

In [5]:
picks_ratios = patch_df.reset_index().set_index(['patchno','champion']).sort_index()
picks_ratios = picks_ratios[['position', 'picks']].set_index('position', append=True).unstack()
totals = picks_ratios.sum(axis=1)

picks_ratios.columns = ['_'.join(col) for col in picks_ratios.columns]
picks_ratios = pd.concat([picks_ratios] + [totals] +\
                 [(picks_ratios[col]/totals) for col in picks_ratios.columns], axis = 1)

new_cols = {i+1:col.replace('picks','ratio') for i,col in enumerate(picks_ratios.columns[:5])}
new_cols[0] = 'totals'
picks_ratios.rename(columns=new_cols, inplace=True)
picks_ratios.sample(5)

,,picks_ADC,picks_Jungle,picks_Middle,picks_Support,picks_Top,totals,ratio_ADC,ratio_Jungle,ratio_Middle,ratio_Support,ratio_Top
patchno,champion,,,,,,,,,,,
6.08,Elise,NaN,21.0,NaN,NaN,NaN,21.0,NaN,1.0,NaN,NaN,NaN
7.12,Rengar,NaN,6.0,NaN,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN
7.01,Twisted Fate,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN
6.18,Twisted Fate,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
5.24,Azir,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN


**Merge Bans**

---

Add the bans df, drop any champion that wasn't picked, and replace NaN's in ban count (champions picked but never banned) with 0. Then add the product of bans and pick ratio to weighted picks for each position.

**Worth Noting:** There are cases where champions were banned but never picked, that information would be of use if I was examining how priority bans impact a team's chances of success, but since I'm interested in the performance of champions in each patch-position, champions that never played are not of interest and would gum up my calculations.

In [6]:
picks_and_bans = pd.concat([picks_ratios.sort_index(), bans.sort_index()],axis = 1)
picks_and_bans.dropna(subset=['totals'], inplace = True)
picks_and_bans.fillna(value={'ban_count':0, 'ban_percent':0}, inplace=True)

for pos in [col.replace('picks_','') for col in picks_and_bans.columns if 'picks' in col]:
    picks_and_bans['weighted_picks_' + pos] = \
    picks_and_bans['picks_'+ pos] + picks_and_bans['ratio_' + pos]*picks_and_bans['ban_count']

**Melt Bans**

---

Melt all those columns back down into rows, fix column labels, and reindex by patch-position-champion.

In [7]:
weighted_picks = pd.melt(picks_and_bans[picks_and_bans.columns[-5:]].reset_index(),id_vars=['patchno','champion'],
        value_vars=list(picks_and_bans.columns[-5:]), var_name = 'position', 
                        value_name = 'weighted_picks').set_index(['patchno','champion']).dropna(subset=['weighted_picks'])

weighted_picks['position'] = weighted_picks['position'].apply(lambda s: s.replace('weighted_picks_',''))

patch_df = patch_df.reset_index().set_index(['patchno','position','champion']).sort_index()
picks_df = pd.concat([patch_df, weighted_picks.reset_index().set_index(['patchno','position','champion']).sort_index()], axis =1)
picks_df.sample(3)

,,,picks,weighted_picks
patchno,position,champion,,
7.03,Jungle,Gragas,2,2.333333
6.08,Middle,Ekko,4,4.800000
6.12,Top,Swain,14,28.518519


**Meta World Peace**

 ---

With each patch update to League of Legends, there comes a new established *meta* - a consensus among teams of what the superior items and champions are. This is mostly an objective calculation of which items are most gold efficient and which champions have the best base power, scaling, or benefit from the current item meta. Dominant playstyles also tend to emerge with each patch, but this is less quantifiable and often the result of trial and error. Hopefully this analysis will ultimately lead to a more certain and immediate determination of a patch's dominant playstyle through understanding what classifications of champions are currently in the meta.

But to decide which champions are currently in the meta, there needs to be a heuristic for assigning pick-level labels based on the distribution of `weighted_picks` for each position in each patch. First, look through the weighted picks for two positions in two patches.

To find universal thresholds for binning a pick-level categorical. Since the counts vary wildly for each series, they'll be scaled by MinMax, MaxAbs, Robust, and Standard

This section has a lot of 'wall-of-text'. It's a necessary evil to make a ground-level inspection of the data in order to understand the methodology for binning.

In [8]:
for p in picks_df.reset_index(level='champion').sample(4,random_state=1).index:
    print(scale_counts(picks_df.reset_index(level='champion'), p).round({'w_pick':1}).head(10))


 ('6.16', 'Middle')
   w_pick  MinMaxScaler  MaxAbsScaler  RobustScaler  StandardScaler
0    33.0         1.000         1.000         3.073           2.325
1    30.0         0.906         0.909         2.760           2.037
2    21.0         0.625         0.636         1.823           1.174
3    19.0         0.562         0.576         1.615           0.982
4     8.0         0.219         0.242         0.469          -0.073
5     6.9         0.185         0.209         0.355          -0.177
6     4.0         0.094         0.121         0.052          -0.456
7     4.0         0.094         0.121         0.052          -0.456
8     3.0         0.062         0.091        -0.052          -0.552
9     3.0         0.062         0.091        -0.052          -0.552

 ('7.08', 'Top')
   w_pick  MinMaxScaler  MaxAbsScaler  RobustScaler  StandardScaler
0    54.0         1.000         1.000         1.149           1.723
1    50.0         0.925         0.926         1.040           1.520
2    49.8

Standard and Robust look like poor choices, as there isn't really consistency to what levels of picks have what values. For some of them the meta cutoff is at 0, for some it's at 1, for others 1.5.

Because MinMax is sensitive to minimum values and I would want a series of nearly uniform distribuation to all be called meta rather than be binned. So MaxAbs it is.

Looking at the MaxAbs in the above, it appears that 0.65 and 0.25 are the threshholds I'm looking for. Anything above 0.65 can be considered meta, anything below 0.25 off-meta, and the in-betweens low-priority. But first that should be tested against edge cases, by looking at the patch-position counts with the widest and narrowest spread.

In [9]:
pick_sd = picks_df.reset_index(level='champion').weighted_picks.std(ddof=0,level=[0,1])
print('Min SD: {:0.3f} at index: {}\nMax SD: {:0.3f} at index: {}'.
      format(pick_sd.min(), pick_sd.idxmin(), pick_sd.max(), pick_sd.idxmax()))

Min SD: 6.023 at index: ('5.24', 'Middle')
Max SD: 77.976 at index: ('7.11', 'Jungle')


In [10]:
print(scale_counts(picks_df, ('5.24', 'Middle',slice(None))).round({'w_pick':1}).head(10))
print(scale_counts(picks_df, ('7.11', 'Jungle',slice(None))).round({'w_pick':1}).head(10))


 ('5.24', 'Middle')
              w_pick  MinMaxScaler  MaxAbsScaler  RobustScaler  StandardScaler
champion                                                                      
Gangplank       24.0         1.000         1.000         3.294           3.009
Lulu            16.0         0.652         0.667         2.039           1.681
Viktor          15.0         0.609         0.625         1.882           1.515
LeBlanc         13.0         0.522         0.542         1.569           1.183
Lissandra       11.1         0.441         0.464         1.277           0.875
Lux              9.0         0.348         0.375         0.941           0.519
Twisted Fate     7.0         0.261         0.292         0.627           0.187
Ryze             6.4         0.235         0.267         0.533           0.087
Ezreal           6.0         0.217         0.250         0.471           0.021
Kassadin         4.8         0.165         0.200         0.282          -0.178

 ('7.11', 'Jungle')
          

The bins aren't perfect, and there are a couple questionable cutoffs for each of the edge cases. But for lack of a better option, they'll have to do.

**Bringing It All Together / Assemble Voltron**

Now that there's a method for classifying pick-levels, the categorical pick_level can be created (along with a few friends).

In [11]:
picks_df['pick_percent'] = picks_df.picks / picks_df.groupby(level=['patchno', 'position'])['picks'].transform(sum)
picks_df['w_pick_percent'] = picks_df.weighted_picks / picks_df.groupby(level=['patchno', 'position'])['weighted_picks'].transform(sum)
picks_df['scaled_picks'] = picks_df.groupby(level=['patchno', 'position'])['weighted_picks'].transform(lambda x: maxabs_scale(x.astype(float)))
picks_df['pick_level'] = pd.cut(picks_df.scaled_picks, bins = [-.1,.25,.65,1.1], labels = ['off-meta', 'low-priority', 'meta'])
picks_df.sample(5, random_state=7)

,,,picks,weighted_picks,pick_percent,w_pick_percent,scaled_picks,pick_level
patchno,position,champion,,,,,,
7.06,Middle,LeBlanc,36,99.000000,0.128571,0.156176,0.853448,meta
6.16,Jungle,Hecarim,2,2.000000,0.026316,0.020202,0.066667,off-meta
7.08,Middle,Karma,4,7.052632,0.025641,0.021185,0.105263,off-meta
7.03,Support,Braum,10,10.000000,0.042017,0.022711,0.114943,off-meta
6.16,ADC,Lucian,13,14.000000,0.171053,0.138614,0.518519,low-priority


Right away the benefit of incorporating the bans data is clear, as a pick like Leblanc is in the meta, but may have been called low-priority  or even off-meta with 63 fewer picks. On top of this, all the pick_levels appear to line up with their prevalance. Moving on!

---

### Phase 2: Capture Champion Averages

I want to take game-performance features and concat the appropriate means `patch_df`.

Before doing that though, I'd like to derive a couple of additional features. I want to capture some information on early and mid-game prominence, so I'll use first blood kill-assist-death information to come up with two features 'fbpresence' and 'fb_kda'.

- 'fbpresence' -- will simply be a 1 for if the champion got or assisted first blood, and a -1 if they gave up first blood.
- 'fb_kda' -- I'll calculate by $fb\_kda = \frac {\sum kills + \sum assists}{max(1, \sum deaths)} $ on the grouped data.
- 'fb_early' -- *A measure of level 1 & 2 kill threat for each champion*  
> if fbpresence > 0 & fbtime $\leq$ 3; fb_early = 1, else 0

---
I will also want to capture a champion's damage efficiency and mid-game power spike.

- 'dmg_efficiency' $= \frac {dmgtochampsperminute}{earnedgpm}$

    - **NOTE:** This is an imperfect metric that I'll try to improve on at another point. Low skirmish games will make this look like a champion is inefficient with dmg. I could use 'dmgshare', but that would be biased against teams with different numbers of dmg dealers in the composition. Perhaps some combination of the two?


- 'midgame_gold_growth' $= \frac {gold_{minute_{15}}-gold_{minute_{10}}}{5}$, this will help show the rate at which certain champions spike or lag going from early to midgame.

---
For junglers (and possibly top laners) I'll create:
- 'jungle_cspm' $= \frac {cs_{jungle}}{gamelength}$
    - **NOTE:** I'll do this for own and enemy jungle seperately.    

In [12]:
f_players['fbpresence'] = f_players['fb'] + f_players['fbassist'] - f_players['fbvictim']

kda = f_players.reset_index().astype({'champion':'object'})\
        .groupby(['patchno','position','champion'])\
        [['fb','fbassist', 'fbvictim']].agg(sum).dropna(how='all')
        
kda['fb_kda'] = np.maximum(0,(kda.fb + kda.fbassist-1)/np.maximum(1, kda.fbvictim))

picks_df['fb_kda'] = kda['fb_kda']
picks_df['fbpresence'] = f_players.reset_index()\
        .groupby(['patchno','position','champion']).fbpresence.sum().dropna(how='all')

# adjustment for small pick frequency
picks_df['fbpresence'] = (picks_df.fbpresence - np.sign(picks_df.fbpresence))/picks_df.picks
picks_df.sample(5)

,,,picks,weighted_picks,pick_percent,w_pick_percent,scaled_picks,pick_level,fb_kda,fbpresence
patchno,position,champion,,,,,,,,
6.21,Middle,Orianna,7,8.000000,0.109375,0.064749,0.258065,low-priority,0.5,0.000000
6.01,Jungle,Lee Sin,27,36.000000,0.083333,0.080887,0.272727,low-priority,2.5,0.222222
7.01,Support,Taric,10,15.000000,0.025126,0.020155,0.080645,off-meta,0.0,0.000000
6.04,ADC,Varus,1,1.818182,0.005155,0.005855,0.019139,off-meta,0.0,0.000000
7.06,Top,Fizz,18,70.200000,0.064286,0.104993,0.605172,low-priority,0.0,-0.166667


**NOTE:** I'm changing up my mapping strategy for `fb_early` because I'm dealing with early game interactions. I've taken mean grouped by patch-champ-position before due to:
- Patch sensitive scaling / item interactions
- Resource allocation by position

These issues don't matter as much here and I'm dealing with a rare event (~5% of games). Don't want small sample picks distorting feature. So I'm only going to take the sum grouped by champion.

Again, I'll add a small correction to keep from giving too much influence to randomly picked champions.

In [13]:
f_players['fb_early'] = 0
f_players.loc[(f_players.fbpresence > 0) & (f_players.fbtime <=3),'fb_early'] = 1
picks_df = picks_df.reset_index().merge(f_players.reset_index().groupby('champion').fb_early\
               .agg(['sum','count']).add_prefix('fb_early_').dropna(how='any')\
               .reset_index(),how='left', on='champion').sort_values(by='fb_early_sum',ascending=False)\
    .set_index(['patchno','position','champion'])

picks_df['fb_early'] = np.maximum(0,picks_df.fb_early_sum - 1)/picks_df.fb_early_count

picks_df.drop(['fb_early_sum','fb_early_count'],axis=1,inplace=True)

**Other Features**

---
Next add dmg_efficiency, midgame_gold_growth, and jungle cspm. Attach these to f_players and bring them into picks_df later.

In [14]:
f_players['dmg_efficiency'] = f_players.dmgshare / f_players.earnedgoldshare

f_players['gold_growth_p1'] = (f_players.goldat10 - 500)/10
f_players['gold_growth_p2'] = (f_players.goldat15 - f_players.goldat10)/5
f_players['gold_growth_p3'] = (f_players.totalgold - f_players.goldat15)/(f_players.gamelength - 15)
f_players['gd_swing_10-15'] = (f_players.gdat15 - f_players.gdat10)/5

f_players['ownjungle_cspm'] = f_players.monsterkillsownjungle / f_players.gamelength
f_players['enemyjungle_cspm'] = f_players.monsterkillsenemyjungle / f_players.gamelength

**Bringing It All Together**

---

Finally, add all those averages to `picks_df`!

In [16]:
champ_feats = ['fbarontime', 'dmgtochampsperminute', 'ownjungle_cspm',
               'enemyjungle_cspm','earnedgoldshare', 'dmg_efficiency', 'cspm', 'gdat10',
               'gold_growth_p1', 'gold_growth_p2','gold_growth_p3', 'gd_swing_10-15', 'xpat10', 'xpdat10']

picks_df = pd.concat([picks_df,\
                      f_players.groupby(['patchno', 'position', 'champion'])[champ_feats].mean().dropna(how='all')]\
                     , axis=1)

# correcting for small sample sizes
picks_df = pd.concat([picks_df, f_players.groupby(['patchno', 'position', 'champion'])['result','fbaron']\
    .agg(lambda x: np.maximum(0,x.sum()-1)/x.count()).dropna(how='all')],axis=1)

picks_df.sample()

,,,picks,weighted_picks,pick_percent,w_pick_percent,scaled_picks,pick_level,fb_kda,fbpresence,fb_early,fbarontime,...,cspm,gdat10,gold_growth_p1,gold_growth_p2,gold_growth_p3,gd_swing_10-15,xpat10,xpdat10,result,fbaron
patchno,position,champion,,,,,,,,,,,,,,,,,,,,,
6.03,Top,Ekko,2,2.0,0.008621,0.005516,0.030303,off-meta,0.0,0.0,0.053537,23.685092,...,7.921493,87.0,246.0,424.5,394.516516,44.6,4228.0,95.0,0.0,0.0


### Phase 3: Calculate Average Win and Loss Times
---

This will help to quantify power peaks and troughs.

In [17]:
picks_df['win_time'] = f_players[f_players.result == 1].groupby(['patchno', 'position', 'champion'])['gamelength'].mean()
picks_df['loss_time'] = f_players[f_players.result == 0].groupby(['patchno', 'position', 'champion'])['gamelength'].mean()

patch_lengths = f_players.groupby('patchno')[['gamelength']].mean()
picks_df.reset_index(level=['position','champion'], inplace=True)
picks_df['patch_lengths'] = patch_lengths
picks_df.set_index(['position','champion'], append=True, inplace=True)

picks_df.loc[picks_df.win_time.isnull(),'win_time'] = picks_df.patch_lengths
picks_df.loc[picks_df.loss_time.isnull(),'loss_time'] = picks_df.patch_lengths
picks_df['delay_need'] = picks_df.win_time - picks_df.loss_time
picks_df.drop('patch_lengths', axis=1, inplace=True)

### Phase 4: Position Specific Features

---

Really, this just applies to supports, which are hard to really evaluate because they dont really do damage or farm and those are the main in-game data points beyond warding. So tacking the cs, farming, and damage values of each support's lane buddy onto their row. This will help give information on what kind of early game power supports have in their lanes.

In [18]:
#support will need feature information of their AD carries:
ad_features = ['gdat10','xpdat10', 'xpat10','earnedgoldshare','dmgtochampsperminute', 'dmg_efficiency',
                'cspm', 'gold_growth_p1', 'gold_growth_p2','gold_growth_p3', 'gd_swing_10-15']
support_features = ['patchno','position','champion']

sup_df = f_players[f_players.position == 'Support'].reset_index().set_index(['gameid','team']).sort_index()[support_features]
ad_df = f_players[f_players.position == 'ADC'].reset_index().set_index(['gameid','team']).sort_index()[ad_features]

support_impact_df = pd.concat([sup_df, ad_df.rename(columns={col:'adc_'+col for col in ad_features})], axis = 1)
support_impact_df.reset_index(inplace=True, drop=True)

support_impact_df = support_impact_df.groupby(['patchno','position','champion']).mean().dropna(how='all')

picks_df = pd.concat([picks_df, support_impact_df], axis=1)

Next fix the labels in picks_df so the don't conflict with existing columns when merged with f_players.

In [19]:
new_cols = {col:'champ_' + col for col in picks_df.columns}
picks_df.rename(columns=new_cols, inplace=True)

**Close up shop**

---

Add the reference data to the player data, save it and move on. Don't worry about NaN's, they're just support features absent in other positions. This will be split into a dict of df's in the next section and the NaN's will vanish.

In [22]:
robust_players = pd.merge( f_players,picks_df.reset_index(), how = 'inner', on = ['patchno','position','champion'])
robust_players.set_index(['gameid','player'],inplace=True)
robust_players.sample(5)

,,league,patchno,side,position,team,champion,ban1,ban2,ban3,ban4,...,champ_adc_xpdat10,champ_adc_xpat10,champ_adc_earnedgoldshare,champ_adc_dmgtochampsperminute,champ_adc_dmg_efficiency,champ_adc_cspm,champ_adc_gold_growth_p1,champ_adc_gold_growth_p2,champ_adc_gold_growth_p3,champ_adc_gd_swing_10-15
gameid,player,,,,,,,,,,,,,,,,,,,,,
1002200043,Sneaky,NALCS,7.1,Red,ADC,Cloud9,Varus,Zac,Galio,Kennen,Kha'zix,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1040523,Chei,LCK,6.06,Red,Support,Jin Air Green Wings,Alistar,Nidalee,Kindred,Graves,NaN,...,-44.333333,3186.594203,0.252991,516.699303,1.060754,9.238505,290.075362,409.252174,450.107725,21.26087
1002160237,Wadid,EULCS,7.13,Blue,Support,Roccat,Tahm Kench,Ashe,Braum,Caitlyn,Blitzcrank,...,-21.468750,3175.156250,0.252270,547.104456,1.108137,9.396315,279.737500,388.556250,464.714792,-9.15625
1090069,Ian,LCK,6.11,Blue,Middle,MVP,Vladimir,Azir,Ryze,Karma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1090279,Duke,LCK,6.11,Blue,Top,SK Telecom T1,Trundle,Nidalee,LeBlanc,Vladimir,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


But wait, there's more...

**BONUS FEATURE**:

'*dmg_efficiency*' $= \frac {dmgshare}{goldshare_{earned}}$

&nbsp;

This is an imperfect metric for several reasons:.

1. Low skirmish games champion look inefficient.
1. Inherently low dps champions look inefficient.
1. Alternative - use 'dmgshare' instead - biased against teams with different numbers of dmg dealers in composition. 

&nbsp;

**Alternative:** Slightly better. Still sensitive to 1) but accounts for 2).

*'rel_dmg'* $= \frac {dpm}{\bar{dpm}_{champ}}$

&nbsp;

- Metric compounds outperforming resources with outperforming champion's average dmg.
- Dividing dpm by avg dpm for champion adjusts for low dps champions.
- Dividing dmg share by gold share adjusts for team's expectations of champion dmg.
- Doesn't fully compensate for team composition - e.g., 4 dmg threats will all likely have dmgshare in the 20%'s **hopefully** that means their goldshare is similar, but it's not gaurunteed. 

This is an improvement. Still not ideal.

In [23]:
robust_players['rel_dmg'] = \
(robust_players.dmgtochampsperminute / robust_players.champ_dmgtochampsperminute)

In [24]:
enpickle(picks_df, 'data/picks_df.pickle')
enpickle(robust_players, 'data/robust_players.pickle')